# Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import random

from extrucal.extrusion import throughput_cal

random.seed(0)

# Dataset Generation using `extrucal`

### 1. Preparation of Lists for Arguments

In [2]:
extruder_size = [size for size in range(20, 251, 10)] * 80640
random.shuffle(extruder_size)    # shuffle works in place and return None
len(extruder_size)

1935360

In [3]:
metering_depth_percent = [np.round(percent, 2) for percent in np.arange(0.02, 0.1, 0.01)] * 241920
random.shuffle(metering_depth_percent)
len(metering_depth_percent)

1935360

In [4]:
polymer_density = [density for density in range(800, 1500, 50)] * 138240
random.shuffle(polymer_density)
len(polymer_density)

1935360

In [5]:
screw_pitch_percent = [np.round(pitch, 2) for pitch in np.arange(0.5, 2, 0.1)] * 129024
random.shuffle(screw_pitch_percent)
len(screw_pitch_percent)

1935360

In [6]:
flight_width_percent = [np.round(percent, 2) for percent in np.arange(0.05, 0.2, 0.01)] * 120960
random.shuffle(flight_width_percent)
len(flight_width_percent)

1935360

In [7]:
number_flight = [1, 2] * 967680
random.shuffle(number_flight)
len(number_flight)

1935360

In [8]:
rpm = [r for r in range(0, 100, 5)] * 96768
random.shuffle(rpm)
len(rpm)

1935360

### 2. Preparation of Dataframe

In [9]:
df = pd.DataFrame(
    {"extruder_size": extruder_size,
     "metering_depth_percent": metering_depth_percent,
     "polymer_density": polymer_density,
     "screw_pitch_percent": screw_pitch_percent,
     "flight_width_percent": flight_width_percent,
     "number_flight": number_flight,
     "rpm": rpm}
)

In [10]:
df["metering_depth"] = df["extruder_size"] * df["metering_depth_percent"]
df["screw_pitch"] = df["extruder_size"] * df["screw_pitch_percent"]
df["flight_width"] = df["extruder_size"] * df["flight_width_percent"]

In [11]:
new_col_order = [
    "extruder_size", "metering_depth", "polymer_density", 
    "rpm", "screw_pitch", "flight_width", "number_flight", ]

df = df[new_col_order]
df.head()

,extruder_size,metering_depth,polymer_density,rpm,screw_pitch,flight_width,number_flight
0,80,7.2,1200,95,48.0,12.0,2
1,220,17.6,1300,75,154.0,13.2,1
2,190,7.6,1400,95,323.0,36.1,2
3,190,7.6,800,55,228.0,26.6,1
4,60,1.8,800,10,48.0,10.8,1


In [12]:
%%time

df.loc[:, "throughput"] = df.apply(
    lambda row: throughput_cal(
        row["extruder_size"],
        row["metering_depth"],
        row["polymer_density"],
        row["rpm"],
        row["screw_pitch"],
        row["flight_width"],
        int(row["number_flight"]))*random.uniform(0.95, 1.05), axis=1
)

CPU times: total: 12min 38s
Wall time: 12min 38s


In [13]:
df

,extruder_size,metering_depth,polymer_density,rpm,screw_pitch,flight_width,number_flight,throughput
0,80,7.2,1200,95,48.0,12.0,2,90.381112
1,220,17.6,1300,75,154.0,13.2,1,4321.460806
2,190,7.6,1400,95,323.0,36.1,2,3029.120568
3,190,7.6,800,55,228.0,26.6,1,1005.047866
4,60,1.8,800,10,48.0,10.8,1,2.794134
...,...,...,...,...,...,...,...,...
1935355,40,3.2,1000,30,32.0,4.0,1,9.063403
1935356,180,3.6,800,85,306.0,27.0,2,774.583875
1935357,30,1.8,1100,60,30.0,2.7,1,7.609954
1935358,70,6.3,1000,40,35.0,10.5,2,11.832348


### 3. Save as CSV file

In [14]:
df.to_csv("../data/extrucal_dataset_w_noise.csv", index=False)